#### Generación de tabla "results_country_prod_company"

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
file_date = dbutils.widgets.get("file_date")

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Obtener datos (Silver)

In [0]:
df_movie = spark.read.format("delta").load(f"{silver_folder_path}/movies").filter(f"file_date = '{file_date}'")
df_country = spark.read.format("delta").load(f"{silver_folder_path}/countries")
df_production_country = spark.read.format("delta").load(f"{silver_folder_path}/production_countries").filter(f"file_date = '{file_date}'")
df_production_company = spark.read.format("delta").load(f"{silver_folder_path}/productions_companies").filter(f"file_date = '{file_date}'")
df_movie_company = spark.read.format("delta").load(f"{silver_folder_path}/movies_companies").filter(f"file_date = '{file_date}'")


### Join "country" y "production_country"

In [0]:
country_prod_coun_df= df_country.join(df_production_country,df_country.country_id==df_production_country.country_id,"inner")\
    .select(df_country.country_name,df_country.country_id,df_production_country.movie_id)

### Join "production_country" y "movie_company"

In [0]:
prod_comp_mov_comp_df = df_production_company.join(df_movie_company,df_production_company.company_id==df_movie_company.company_id,"inner")\
    .select(df_production_company.company_name,df_movie_company.company_id,df_movie_company.movie_id)

In [0]:
movie_filter_df = df_movie.filter("year_release_date >= 2010")

In [0]:
results_movies_country_prod_company_df = movie_filter_df.join(country_prod_coun_df, movie_filter_df.movie_id == country_prod_coun_df.movie_id,"inner")\
    .join(prod_comp_mov_comp_df, movie_filter_df.movie_id == prod_comp_mov_comp_df.movie_id,"inner")

In [0]:
df_results = results_movies_country_prod_company_df.select(movie_filter_df.movie_id,"country_id","company_id","title","budget","duration_time","release_date","country_name","company_name").withColumn("created_date",lit(file_date))

In [0]:
results_order_by_df = df_results.orderBy(df_results.title.asc())

#### Escribir resultado en Gold

In [0]:
merge_condition="target.movie_id = source.movie_id  AND target.country_id = source.country_id AND target.company_id = source.company_id AND target.created_date = source.created_date"
merge_delta_lake(results_order_by_df,"movie_gold","results_country_prod_company",gold_folder_path,merge_condition,"created_date")

In [0]:
%sql
SELECT * FROM movie_gold.results_country_prod_company